In [148]:
from sklearn import linear_model
import pickle
from collections import defaultdict
import numpy

with open("Cat_price_filtered_review_train.pkl","rb") as fp:
     cat_dicts = pickle.load(fp)
        
train_cat_dicts = cat_dicts[:600000]
#val_cat_dicts = cat_dicts[500000:600000]
test_cat_dicts = cat_dicts[600000:]

with open("Grid_dict.pkl","rb") as fp:
     grid_dicts = pickle.load(fp)
        
with open("Top_Categories_50.pkl","rb") as fp:
     top_dicts = pickle.load(fp)

In [222]:
cat_dicts[:10]
#grid_dicts[(41.0, -122.0)]

[{'rating': 4.0,
  'reviewerName': 'william spindler',
  'reviewText': 'Best War Wanton soup in Red Bluff',
  'categories': ['Asian Restaurant', 'Chinese Restaurant'],
  'gPlusPlaceId': '106591714648856494903',
  'unixReviewTime': 1394669496,
  'reviewTime': 'Mar 12, 2014',
  'gPlusUserId': '100000032416892623125',
  'gps': (40.179159, -122.236162)},
 {'rating': 5.0,
  'reviewerName': 'william spindler',
  'reviewText': 'Long time favorite Mexican food, always consistent and great tasting.',
  'categories': ['Mexican Restaurant'],
  'gPlusPlaceId': '115827996910815192564',
  'unixReviewTime': 1394669713,
  'reviewTime': 'Mar 12, 2014',
  'gPlusUserId': '100000032416892623125',
  'gps': (40.175064, -122.242574)},
 {'rating': 2.0,
  'reviewerName': 'Richard Yocom',
  'reviewText': 'Open 24 hours is a plus but really slow. Sat in drive thru for 30 minutes. Was not real friendly. Food was ok',
  'categories': ['Restaurant'],
  'gPlusPlaceId': '100073820849130920147',
  'unixReviewTime': 13

In [149]:
users = set([x['gPlusUserId'] for x in train_cat_dicts])
places = set([x['gPlusPlaceId'] for x in train_cat_dicts])

In [145]:
#user_di['110066519980916079964']

In [150]:
mod_train_dicts = []
usersPerPlace = defaultdict(set) # Maps an item to the users who rated it
placesPerUser = defaultdict(set) # Maps a user to the items that they rated

for d in train_cat_dicts:
    user = d['gPlusUserId']
    place = d['gPlusPlaceId']
    usersPerPlace[place].add(user)
    placesPerUser[user].add(place)
    #r = int(d['rating'])
    #d['rating'] = r
    mod_train_dicts.append(d)

In [151]:
mod_train_dicts[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125',
 'gps': (40.179159, -122.236162)}

In [152]:
mod_test_dicts = []
usersPerPlace_test = defaultdict(set) # Maps an item to the users who rated it
placesPerUser_test = defaultdict(set) # Maps a user to the items that they rated

for d in test_cat_dicts:
    user = d['gPlusUserId']
    place = d['gPlusPlaceId']
    usersPerPlace_test[place].add(user)
    placesPerUser_test[user].add(place)
    #r = int(d['rating'])
    #d['rating'] = r
    mod_test_dicts.append(d)

In [153]:
mod_test_dicts[0]

{'rating': 4.0,
 'reviewerName': 'Alyssa Blaylock',
 'reviewText': 'Best Food In town. period.',
 'categories': ['Barbecue Restaurant'],
 'gPlusPlaceId': '101528116980929093146',
 'unixReviewTime': 1338420798,
 'reviewTime': 'May 30, 2012',
 'gPlusUserId': '114572295318502583651',
 'gps': (44.637578, -123.10424)}

In [52]:
#usersPerPlace['106591714648856494903']

{'100000032416892623125', '100660899335968702364', '109681769048735571251'}

In [154]:
one_hot_cat = dict()
zero = [0]*49
j = 0
one_hot_cat["['American Restaurant']"] = zero
for i in top_dicts[1:]:
    one_hot_cat[str(i)] = zero[:j] + [1] + zero[j+1:]
    j = j + 1

In [156]:
min_lat = 20
max_lat = 60

min_long = -125
max_long = -60

def grid_id(lat,lon,lat_lines,long_lines):
    if lat < lat_lines.min() or lat > lat_lines.max() or lon < long_lines.min() or lon > long_lines.max():
        return (-1,-1)
    else:
        grid_lat = lat_lines[numpy.searchsorted(lat_lines,lat)]
        grid_long = long_lines[numpy.searchsorted(long_lines,lon)]
    return (grid_lat,grid_long)

lat_lines = numpy.linspace(min_lat,max_lat,num=max_lat-min_lat +1)
long_lines = numpy.linspace(min_long,max_long,num=max_long-min_long+1)

In [228]:
max_len = 0
max_place = 0
max_user = 0
max_grid = 0
sum_len = 0
for i in mod_train_dicts:
    if(i['reviewText'] != None and len(i['reviewText']) > max_len):
        max_len = len(i['reviewText'].split())
        sum_len = sum_len + len(i['reviewText'].split())

    if(len(usersPerPlace[i['gPlusPlaceId']]) > max_place):
        max_place = len(usersPerPlace[i['gPlusPlaceId']])
        
    if(len(placesPerUser[i['gPlusUserId']]) > max_user):
        max_user = len(placesPerUser[i['gPlusUserId']])
        
    if(len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)]) > max_grid):
        max_grid = len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)])
        
avg_len = sum_len/len(mod_train_dicts)

In [229]:
x_train_feat = []
y_train_lab = []
for i in mod_train_dicts:
    if(i['reviewText'] != None):
        x_train_feat.append([1] + [len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)])/max_grid] + [len(i['reviewText'].split())/max_len] + one_hot_cat[str(i['categories'])] + [len(usersPerPlace[i['gPlusPlaceId']])/max_place] + [len(placesPerUser[i['gPlusUserId']])/max_user])
    else:
        x_train_feat.append([1] + [len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)])/max_grid] + [avg_len/max_len] + one_hot_cat[str(i['categories'])] + [len(usersPerPlace[i['gPlusPlaceId']])/max_place] + [len(placesPerUser[i['gPlusUserId']])/max_user])        
    y_train_lab.append(i['rating'])

In [230]:
x_test_feat = []
y_test_lab = []
for i in mod_test_dicts:
    if(i['reviewText'] != None):
        x_test_feat.append([1] + [len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)])/max_grid] + [len(i['reviewText'].split())/max_len] + one_hot_cat[str(i['categories'])] + [len(usersPerPlace[i['gPlusPlaceId']])/max_place] + [len(placesPerUser[i['gPlusUserId']])/max_user])
    else:
        x_test_feat.append([1] + [len(grid_dicts[grid_id(i['gps'][0], i['gps'][1], lat_lines, long_lines)])/max_grid] + [avg_len/max_len] + one_hot_cat[str(i['categories'])] + [len(usersPerPlace[i['gPlusPlaceId']])/max_place] + [len(placesPerUser[i['gPlusUserId']])/max_user])        
    y_test_lab.append(i['rating'])

In [231]:
# Training model and finding theta
theta, residuals, rank, s = numpy.linalg.lstsq(x_train_feat, y_train_lab)


<ipython-input-231-56727cb93e76>:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta, residuals, rank, s = numpy.linalg.lstsq(x_train_feat, y_train_lab)


In [242]:
def MSE(y, pred):
    sq_err = 0
    for i in range(len(y)):
        sq_err = sq_err + (y[i] - pred[i])**2
    mse = (1/len(y))*sq_err
    return mse

# Train Predicting
pred = []
for i in x_train_feat:
    pred.append(numpy.dot(theta, i))
    
for i in range(len(pred)):
    if pred[i] > 5:
        pred[i] = 5
    if pred[i] < 0:
        pred[i] = 0
    
mse = MSE(y_train_lab, pred)
print("Train MSE: ", mse)

# Val Predicting
pred = []
for i in x_test_feat:
    pred.append(numpy.dot(theta, i))

for i in range(len(pred)):
    if pred[i] > 5:
        pred[i] = 5
    if pred[i] < 0:
        pred[i] = 0

mse = MSE(y_test_lab, pred)
print("Val MSE: ", mse)

Train MSE:  1.3635293792313805
Val MSE:  1.3326082139137596


In [256]:
# XGBoost

import xgboost as xgb

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=500, learning_rate=0.05, gamma = 5, reg_lambda = 5,subsample=0.3, colsample_bytree=0.5).fit(numpy.asarray(x_train_feat), numpy.asarray(y_train_lab))
gbm_train_pred = gbm.predict(numpy.asarray(x_train_feat))

gbm_val_pred = gbm.predict(numpy.asarray(x_test_feat))

[14:35:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [257]:
'''for i in range(len(gbm_train_pred)):
    if gbm_train_pred[i] > 5:
        gbm_train_pred[i] = 5
    if gbm_train_pred[i] < 0:
        gbm_train_pred[i] = 0'''

mse = MSE(y_train_lab, gbm_train_pred)
print("Train MSE: ", mse)

'''for i in range(len(gbm_val_pred)):
    if gbm_val_pred[i] > 5:
        gbm_val_pred[i] = 5
    if gbm_val_pred[i] < 0:
        gbm_val_pred[i] = 0'''
        
mse = MSE(y_test_lab, gbm_val_pred)
print("Val MSE: ", mse)

Train MSE:  2.254835
Val MSE:  2.4842000000000004
